# Import Some Libraries

In [1]:
import os
import time
from datetime import datetime 

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import MaxNLocator
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

# Merge Reference Monitor data with start-up's sensor data

In [2]:
def data_merge(name_list):
    '''
    merge reference monitor data with start-up's data into a single dataframe
    Input:- name of different locations
    '''
    raw_data_path = r'C:\Users\ACER ONE\PM2.5 Paper Code\paper_code\data\raw_data'
    
    for name in name_list:
        
        # for Reference Monitor Data
        df_name = pd.read_excel(raw_data_path+'/Reference_'+name+'.xlsx')
        df_name = df_name.iloc[15:]
        df_name.columns = df_name.iloc[[0]].values.flatten().tolist()
        df_name = df_name.drop(15).reset_index(drop=True)
        df_name = df_name[['From Date','PM2.5']]
        df_name = df_name.rename(columns = {'From Date':'Date','PM2.5':'PM25'})
        df_name = df_name.replace('None',np.nan)
        df_name['Date'] = pd.to_datetime(df_name.Date,dayfirst = True)
        df_name = df_name[df_name.Date < '2021-02-01']
        
        # for Start-up's data
        lower_name = str.lower(name)
        df_startup = pd.read_csv(raw_data_path+'\StartupA_'+lower_name+'.csv')
        df_startup = df_startup[['dt_time','pm2.5cnc','pm10cnc','temp','rh']]
        df_startup = df_startup.rename(columns = {'dt_time':'Date','pm2.5cnc':'pm25cnc'})
        df_startup['Date'] = pd.to_datetime(df_startup.Date,dayfirst = True)
        df_startup = df_startup[df_startup.Date < '2021-02-01']
        df_final = pd.merge(df_name,df_startup,on = 'Date', how = 'outer')
        
        # merge dataset path
        merge_data_path = r'C:\Users\ACER ONE\PM2.5 Paper Code\paper_code\data\ref_startupA_data'
        
        # Saving the merge dataset
        df_final.to_csv(merge_data_path +  '/ref_startupA_' + name + '.csv', index=False)

# Input - Output pairs formation for base features

In [3]:
def base_features(name_list):
    '''
    Input:- merge dataset i.e. startup's data with reference monitor data
    Output:- get clean data pairs of startup's data as input and reference data as output 
    by removing missing values and outliers points
    '''
    
    merge_data_path = r'C:\Users\ACER ONE\PM2.5 Paper Code\paper_code\data\ref_startupA_data'
    
    for name in name_list:
        df_name = pd.read_csv(merge_data_path + '/ref_startupA_'+name+'.csv')
        
        # replace outliers point as missing values
        temp_max = 50
        rh_max = 100
        
        for i in range(len(df_name)):
            if df_name['temp'][i] > temp_max:
                df_name['temp'][i] = np.nan
                
        for i in range(len(df_name)):
            if df_name['rh'][i] > rh_max:
                df_name['rh'][i] = np.nan
        
        # remove missing values
        df_name = df_name.replace(0,np.nan)
        df_name = df_name.mask(df_name.eq('None')).dropna()
        df_name = df_name.reset_index(drop=True)
        
        # base features path
        base_data_path = r'C:\Users\ACER ONE\PM2.5 Paper Code\paper_code\data\base_features'
        
        # save base features
        df_name.to_csv(base_data_path +  '/base_innu_outu_startupA_' + name + '.csv', index=False)

# Input - Output pair formation for base+derived features

In [4]:
def base_derived_features(name_list):
    '''
    Input:- merge dataset i.e. startup's data with reference monitor data
    
    Output:- get clean data pairs of startup's raw data and some derived features as input and 
    reference data as output by removing missing values and outliers points
    '''
    merge_data_path = r'C:\Users\ACER ONE\PM2.5 Paper Code\paper_code\data\ref_startupA_data'
    
    for name in name_list:
        df_name = pd.read_csv(merge_data_path + '/ref_startupA_'+name+'.csv')
        
        df_name.Date = pd.to_datetime(df_name.Date,dayfirst=True)
        
        # replace outliers point as missing values
        temp_max = 50
        rh_max = 100
        
        for i in range(len(df_name)):
            if df_name['temp'][i] > temp_max:
                df_name['temp'][i] = np.nan
                
        for i in range(len(df_name)):
            if df_name['rh'][i] > rh_max:
                df_name['rh'][i] = np.nan
        
        # hour of the day 
        df_name['hour'] = df_name.Date.dt.strftime('%H').astype(int)+1
        
        # due to periodicity
        df_name['hour_sin'] = np.sin(2*np.pi*df_name['hour']/24)
        df_name['hour_cos'] = np.cos(2*np.pi*df_name['hour']/24)
        
        df_name['rh_11'] = (df_name['rh']*df_name['rh'])/(df_name['rh'] - 1)
        
        # effective time lags based features
        lags = [25,24,23,3,2,1]
        for i in lags:
            
            # for pm25cnc
            df_name['(PM25)t-'+str(i)] = df_name['pm25cnc'].shift(i)
            
            # for pm10cnc 
            df_name['(PM10)t-'+str(i)] = df_name['pm10cnc'].shift(i)

        # multiplication based features
        df_name['p1p2'] = df_name['pm25cnc']*df_name['pm10cnc']
        df_name['p1r'] = df_name['pm25cnc']*df_name['rh']
        df_name['p1t'] = df_name['pm25cnc']*df_name['temp']
        df_name['p2r'] = df_name['pm10cnc']*df_name['rh']
        df_name['p2t'] = df_name['pm10cnc']*df_name['temp']
        df_name['p1rt'] = df_name['pm25cnc']*df_name['rh']*df_name['temp']
        df_name['p2rt'] = df_name['pm10cnc']*df_name['rh']*df_name['temp']
        df_name['p1p2rt'] = df_name['pm25cnc']*df_name['pm10cnc']*df_name['rh']*df_name['temp']
        
        # remove missing values
        df_name = df_name.replace(0,np.nan)
        df_name = df_name.mask(df_name.eq('None')).dropna()
        df_name = df_name.reset_index(drop=True)
        
        # base+derived features path
        base_derived_data_path = r'C:\Users\ACER ONE\PM2.5 Paper Code\paper_code\data\base_derived_features'
        
        # save base+derived features
        df_name.to_csv(base_derived_data_path +  '/base_derived_innu_outu_startupA_'+ name + '.csv', index=False)   

# Save data

In [5]:
name_list = ['Airport','Borivali','Kalyan','Mahape','Nerul','Powai','Vileparle','Worli']

# get the merge dataset (reference monitor with startup's data)
data_merge(name_list)

# get base features
base_features(name_list)

# get base+derived features
base_derived_features(name_list) 